# Desafio 2:

1. Crear sus propios vectores con Gensim basado en lo visto en clase con otro artista del dataset Songs.

2. Elegir términos de interés y buscar términos más similares y menos similares.

3. Realizar una reduccion de dimensionalidad a los embeddings, llevándolos a 2 dimensiones. Graficar los embeddings proyectados y seleccionar una cantidad de términos (variable MAX_WORDS) de forma tal que la visualización sea adecuada.

4. Inspeccionar el grafico y buscar pequeños grupos de palabras que puedan formarse. Interpretarlos e intentar obtener conclusiones. En lo posible, acompañar los grupos de palabras con capturas (y pegarlas en celdas de texto)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
try:
  from gensim.models import Word2Vec
except:
  !pip install gensim
  from gensim.models import Word2Vec

## 1. Eligo el Dataset de Bob Marley:

In [ ]:
# Descargar la carpeta de dataset
import os
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/bob-marley.txt', sep='/n', header=None)
df.head()

C:\Users\jonat\AppData\Local\Temp\ipykernel_29560\3034298888.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('songs_dataset/bob-marley.txt', sep='/n', header=None)


,0
0,"""Don't worry about a thing,"
1,'Cause every little thing gonna be all right.
2,"Singin': ""Don't worry about a thing,"
3,"'Cause every little thing gonna be all right!""..."
4,"Smiled with the risin' sun,"


In [ ]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 2218


### Preprocesamiento

In [ ]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [ ]:
# Demos un vistazo
sentence_tokens[:2]

[["don't", 'worry', 'about', 'a', 'thing'],
 ["'cause", 'every', 'little', 'thing', 'gonna', 'be', 'all', 'right']]

### Crear los vectores (word2vec)

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [ ]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [ ]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 2218


In [ ]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 532


### Entrenar embeddings

In [ ]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 150191.34375
Loss after epoch 1: 93119.03125
Loss after epoch 2: 92445.84375
Loss after epoch 3: 92579.71875
Loss after epoch 4: 92034.40625
Loss after epoch 5: 88941.90625
Loss after epoch 6: 82685.1875
Loss after epoch 7: 79544.375
Loss after epoch 8: 75713.25
Loss after epoch 9: 72145.0
Loss after epoch 10: 70732.3125
Loss after epoch 11: 68090.25
Loss after epoch 12: 62245.375
Loss after epoch 13: 60419.375
Loss after epoch 14: 60237.375
Loss after epoch 15: 59960.375
Loss after epoch 16: 58845.75
Loss after epoch 17: 58794.0
Loss after epoch 18: 58291.75
Loss after epoch 19: 57500.0


(218093, 359260)

## 2. Elijo términos de interés y busco términos más similares y menos similares.

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[('protect', 0.808631181716919),
 ('nobody', 0.8049882054328918),
 ('given', 0.803276538848877),
 ("you've", 0.7994995713233948),
 ('really', 0.7939799427986145),
 ("feelin'", 0.7812823057174683),
 ('message', 0.7800707221031189),
 ('ask', 0.7799001336097717),
 ('mean', 0.7632033824920654),
 ('door', 0.7456943392753601)]

### Explicación:
En este ejemplo "love" está relacionado con "protect" tal vez por la cación "Positive Vibration" que dice "Jah love, Jah love (protect us)".

Tambien "nobody" puede estar relacionado en la canción "All Day All Night" por la frase "I’m gonna give my love to nobody but you baby"

La palabra "given" por la cación Stand Alone" en la frase "And while you're takin' true love, you given the blame"

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[('ganja', -0.20341800153255463),
 ('de', -0.20613440871238708),
 ("'pon", -0.21176497638225555),
 ('walk', -0.2134210169315338),
 ("'ypocrites", -0.2166365534067154),
 ('a', -0.2277471423149109),
 ('wall', -0.22777627408504486),
 ("smoke'a", -0.22913460433483124),
 ('gone', -0.23127853870391846),
 ('fly', -0.23372715711593628)]

### Explicación:

De las palabras menos similares tenemos sabemos que no tienen ninguna relación con "love" lo cual tiene sentido.

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["vibration"], topn=10)

[('positive', 0.9743016958236694),
 ("i'n'i", 0.9051023125648499),
 ('rastaman', 0.8862149119377136),
 ('everything', 0.8744521737098694),
 ('african', 0.8633027672767639),
 ('within', 0.8555753231048584),
 ('fadeout', 0.8545414209365845),
 ('him', 0.8513715267181396),
 ('blowing', 0.8508684635162354),
 ('human', 0.8499391078948975)]

### Explicación:

Las palabras mas comunes se relacion en la canción "positve vibration" en las frase: 
- "positive vibration"
- "I and I vibration, yeah"
- "rastaman vibration, yeah"


In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(negative=["vibration"], topn=10)

[('there', -0.36773914098739624),
 ('sweet', -0.40117713809013367),
 ('a', -0.40556636452674866),
 ('more', -0.4084632396697998),
 ('place', -0.40855178236961365),
 ('dread', -0.4163464903831482),
 ('slogans', -0.42235392332077026),
 ('from', -0.42497333884239197),
 ('much', -0.4250175952911377),
 ('take', -0.4260817766189575)]

## 3. Realizar una reduccion de dimensionalidad a los embeddings, llevándolos a 2 dimensiones. Graficar los embeddings proyectados y seleccionar una cantidad de términos (variable MAX_WORDS) de forma tal que la visualización sea adecuada.

In [ ]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=50
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="vscode") # esto para plotly en colab, pero para vscode es mejor

In [ ]:
import plotly.io as pio
pio.renderers
pio.renderers.default

'vscode'